<b>Partie IA</b>

Imports

In [11]:
# Importation des bibliothèques nécessaires
import pandas as pd  # Pour manipuler les données sous forme de DataFrame
import warnings  # Pour gérer les avertissements

# Importation des modules de Scikit-learn pour la transformation des colonnes, pipelines, modèles et évaluation
from sklearn.compose import ColumnTransformer  # Pour appliquer des transformations différentes sur les colonnes
from sklearn.pipeline import Pipeline  # Pour créer des pipelines de transformations et de modèles
from sklearn.preprocessing import StandardScaler  # Pour le prétraitement des données
from sklearn.model_selection import train_test_split  # Pour la division des données et la recherche des hyperparamètres
from sklearn.metrics import accuracy_score, classification_report  # Pour évaluer les performances du modèle
from catboost import CatBoostClassifier  # Classifieur CatBoost

# Ignorer certains avertissements
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn



In [12]:
df_tr = pd.read_csv(r'C:\Users\colin\Documents\ESEO\E5\docsPFE\creditcard_2023.csv')

L'étude des variables sur la partie Big Data nous a permis de voir que certaines corrélations de variables sont plus importantes que d'autres dans le dataset pour les transactions frauduleuse. Car nous ne pouvont pas être 100% sûrs de pouvoir enelever celles qui n'ont pas l'air importantes, nous les gardons.

On teste plusieurs méthodes d'intelligence artificielle pour en trouver une qui est à la fois rapide et efficace.

*Apprentissage par renforcement*

In [13]:
# Séparation des données en caractéristiques (X) et cible (y)
X = df_tr.drop('Class', axis=1)
y = df_tr['Class']

In [14]:
# Définir les caractéristiques numériques (exclure les colonnes catégorielles)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [15]:
# Définir le pipeline de transformation pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

StandardScaler() est un transformateur qui normalise les données en les centrant sur la moyenne 0 et en les réduisant à une échelle unitaire (écart-type 1). Cela est souvent nécessaire avant de passer à des algorithmes d'apprentissage automatique, car certains modèles sont sensibles à l'échelle des données (par exemple, les modèles basés sur la distance comme la régression logistique, SVM, etc.).

In [16]:
# Définir la transformation par colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

In [17]:
# Définir le modèle
model = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', CatBoostClassifier(verbose=False))])

In [18]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
# Entraîner le modèle
model.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['id', 'V1', 'V2', 'V3', 'V4',
                                                   'V5', 'V6', 'V7', 'V8', 'V9',
                                                   'V10', 'V11', 'V12', 'V13',
                                                   'V14', 'V15', 'V16', 'V17',
                                                   'V18', 'V19', 'V20', 'V21',
                                                   'V22', 'V23', 'V24', 'V25',
                                                   'V26', 'V27', 'V28',
                                                   'Amount'])])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x0000019CC1C17A30>)])

In [20]:
# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

In [21]:
# Calculer l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9996570705027874


In [22]:
# Afficher le rapport de classification avec couleurs et en-tête
from colorama import Fore, Back, Style
print("\n" + Back.BLUE + Fore.WHITE + "Classification Report" + Style.RESET_ALL)
report = classification_report(y_test, y_pred, output_dict=True)
for key, value in report.items():
    if key in ['0', '1']:  
        color = Fore.GREEN if value['precision'] > 0.8 else Fore.RED
        print(f"Class {key}:")
        print(f"  Precision: {color}{value['precision']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['recall'] > 0.8 else Fore.RED
        print(f"  Recall: {color}{value['recall']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['f1-score'] > 0.8 else Fore.RED
        print(f"  F1-score: {color}{value['f1-score']:.2f}{Style.RESET_ALL}")
        print(f"  Support: {value['support']}")
    else:
        print(key + ":", value)


Classification Report
Class 0:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56750.0
Class 1:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56976.0
accuracy: 0.9996570705027874
macro avg: {'precision': 0.9996569545708367, 'recall': 0.9996571914660413, 'f1-score': 0.9996570692311153, 'support': 113726.0}
weighted avg: {'precision': 0.999657078158671, 'recall': 0.9996570705027874, 'f1-score': 0.9996570705434346, 'support': 113726.0}


*Forêt aléatoire*

On va donc séparer nos données en un échantillon d’apprentissage pour apprendre notre modèle et un échantillon test pour tester l’efficacité de la forêt aléatoire.

In [40]:
x_train, x_test, y_train, y_test  = train_test_split(X, 
                                                     y, 
                                                     test_size=0.25, 
                                                     random_state=42)

On utilise un échantillon de test représentant 25% des données.

On va utiliser scikit-learn pour construire notre forêt aléatoire, on utilise donc la classe RandomForestClassifier de scikit-learn et on crée un objet à partir de cette classe. C’est à ce moment que nous allons définir ce qu’on appelle les hyperparamètres du modèle. Il s’agit de paramètres tels que le nombre d’arbres dans la forêt qui doivent être définis en amont de l’apprentissage.

In [28]:
modele_rf = RandomForestClassifier(
     n_estimators=100,
     criterion='gini',
     max_depth=None,
     min_samples_split=2,
     min_samples_leaf=1,
     min_weight_fraction_leaf=0.0,
     max_features='sqrt',
     max_leaf_nodes=None,
     min_impurity_decrease=0.0,
     bootstrap=True,
     oob_score=False,
     n_jobs=None,
     random_state=None,
     verbose=0,
     warm_start=False,
     class_weight=None,
     ccp_alpha=0.0,
     max_samples=None,)

Tous ces hyperparamètres ont des valeurs par défaut que nous modifions rarement, mais il est utile de les comprendre. En voici un description rapide :

n_estimators : il s’agit du nombre d’arbres dans la forêt
criterion : il s’agit du critère utilisé pour construire les arbres et séparer les branches des arbres

max_depth : il s’agit de la profondeur maximale des arbres utilisés (le nombre de niveaux dans l’arbre de décision)

min_sample_split : il s’agit du nombre d’échantillons minimal dans une feuille pour refaire une séparation

min_samples_leaf : il s’agit du nombre d’échantillons minimal pour créer une feuille

min_weight_fraction_leaf : il s’agit de la fraction du nombre total d’échantillon minimal pour créer une feuille

max_features : il s’agit du nombre de colonnes sélectionnées pour chaque arbre (par défaut on prend la racine carré du nombre de colonnes)

max_leaf_nodes : il s’agit du nombre maximal de feuilles

min_impurity_decrease : il s’agit de la baisse minimale du critère d’impureté pour faire une séparation

bootstrap : paramètre pour utiliser du bootstrap, si il est à False, le même échantillon est pris pour chaque arbre

n_jobs ; nombre de traitements à effectuer en parallèle
random_state : graine aléatoire

warm_start : ceci permet de repartir du résultat du dernier apprentissage pour faire l’apprentissage

class_weights : il s’agit des poids associés à chaque classe si cela a un sens

max_samples : si vous voulez réduire le nombre d’observations dans vos échantillons bootstrap


Nous avons donc construit un objet forêt aléatoire, nous devons maintenant effectuer l’apprentissage :

In [29]:
modele_rf.fit(x_train, y_train)

RandomForestClassifier()

On peut afficher l’importance des variables de notre modèle de forêt aléatoire :

In [30]:
pd.DataFrame(modele_rf.feature_importances_,
              index = x_train.columns, 
              columns = ["importance"]).sort_values(
     "importance", 
     ascending = False)

,importance
id,0.356352
V10,0.154206
V14,0.113836
V4,0.109211
V17,0.055781
V11,0.042316
V12,0.033157
V16,0.031595
V3,0.017664
V2,0.014934


Une fois notre modèle appris, il faut le valider. Pour cela, on va utiliser les données de test et des indicateurs très classiques.

On compare donc la valeur prédite par le modèle avec la valeur dans les données. Il s’agit ici de comparer la couleur du vin prédite par le modèle et celle observée dans notre échantillon test.

Le premier indicateur utilisé est le pourcentage de bien classé aussi appelé accuracy :

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(f"Le pourcentage de bien classés est de : {accuracy_score(y_test, modele_rf.predict(x_test))*100} %")

Le pourcentage de bien classés est de : 99.98241393379197 %


https://www.stat4decision.com/fr/foret-aleatoire-avec-python/

*Réseau de neuronnes*

In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [44]:
print(type(X))
print(type(y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [47]:
X_np = X.to_numpy()  # Use .to_numpy() or .values to convert to NumPy array
y_np = y.to_numpy()  # Convert Series to NumPy array

# Convert to PyTorch tensors
X_nn = torch.tensor(X_np, dtype=torch.float32)  # Convert features to tensor
y_nn = torch.tensor(y_np, dtype=torch.float32).reshape(-1, 1)  # Convert target to tensor and reshape


In [49]:
model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid())

In [50]:
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


In [51]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [52]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [54]:
n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X_nn), batch_size):
        Xbatch = X_nn[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_nn[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x30 and 8x12)

https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/